## preparation work 

In [1]:
"""
before start 
pip install -U spacy 
pip install regex
pip install nltk(if needed)
pip install json
pip install emoji
pip install textblob

"""
import numpy as np
import json
import pandas as pd
import emoji
import nltk
import spacy
import regex as re
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

In [2]:
nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')


('parser', <spacy.pipeline.pipes.DependencyParser at 0x1fd4c00d828>)

In [3]:
"""
run these code if needed
nltk.download("corpus")
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

"""


'\nrun these code if needed\nnltk.download("corpus")\nnltk.download("stopwords")\nnltk.download(\'punkt\')\nnltk.download(\'wordnet\')\nnltk.download(\'averaged_perceptron_tagger\')\n\n'

# process from json

## load and filter the data from jason

In [4]:
### load json (scot) into a list
raw_data_list = []
with open('sample_data.json','r') as tweets:
    line=tweets.readline()
    for line in tweets:
        if line != '\n':
            dic=json.loads(line)
            raw_data_list.append(dic)

In [5]:
print('the number of tweets get:',len(raw_data_list))

the number of tweets get: 131849


In [6]:
### clean scot topic data
cleaned_data_list=[]
for data in raw_data_list:
    if 'text'in data.keys() and 'truncated' in data.keys():
        if len(data['text'])!=139 and len(data['text'])!=140 and data['lang'] =='en' and len(data['text'])>60:
            cleaned_data_list.append(data)

In [7]:
print('the  number of qualified tweets:',len(cleaned_data_list))

the  number of qualified tweets: 29821


In [8]:
### creating the dataframe
basicdata=[]
for i in range(len(cleaned_data_list)):
    basicdata.append([cleaned_data_list[i]['id'],cleaned_data_list[i]['created_at'],cleaned_data_list[i]['text'],len(cleaned_data_list[i]['text'])])
df=pd.DataFrame(basicdata,columns=['id','time','text','length'])

## extract hashtag

In [9]:
### add hash tag list into the dataframe
hashtag_list=[]
for i in range(0,len(cleaned_data_list)):
    hashtag_each_list=[]
    if len(cleaned_data_list[i]['entities']['hashtags'])!=0:
        for j in range(0,len(cleaned_data_list[i]['entities']['hashtags'])):
            hashtag_each_list.append(cleaned_data_list[i]['entities']['hashtags'][j]['text'])
    
    if hashtag_each_list:
        hashtag_list.append(hashtag_each_list)
    else:
        hashtag_list.append('')

df['hash tags']=hashtag_list

In [10]:
"""
code for deleting df columns

df.drop(labels='hash1',axis=1,inplace=True)

"""

"\ncode for deleting df columns\n\ndf.drop(labels='hash1',axis=1,inplace=True)\n\n"

## extract emoji 

In [11]:
### define a function to extract emojis (return a list)
def extract_emojis(str):
    emoji_list=[]
    for c in str:
        if c in emoji.UNICODE_EMOJI:
            emoji_list.append(c)
    return emoji_list

In [12]:
### extract emoji from the text and load them into the data frame
emoji_lists=[]
for i in range(0,len(df)):
    emoji_lists.append(extract_emojis(df.iloc[i].at['text']))
df['emoji']=emoji_lists

In [13]:
df

,id,time,text,length,hash tags,emoji
0,1236305964870938624,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[Happy, TAEYEON, 태연, 소녀시대, GirlsGeneration, 태연...",[🎀]
1,1236305965395234817,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[Happy, TAEYEON, 태연, 소녀시대, GirlsGeneration, 태연...",[🎀]
2,1236305965546213376,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[Happy, TAEYEON, 태연, 소녀시대, GirlsGeneration, 태연...",[🎀]
3,1236305965718417417,Sat Mar 07 15:01:26 +0000 2020,RT @missradders: Someone needs to hold him to ...,74,,[👗]
4,1236305966292926466,Sat Mar 07 15:01:26 +0000 2020,RT @espn: MEAN MUG BRON 😤\n\nLeBron has been u...,111,,[😤]
...,...,...,...,...,...,...
29816,1236355889751343107,Sat Mar 07 18:19:49 +0000 2020,RT @SteveChopz: Found someone special in Idivi...,77,,[😆]
29817,1236355889998651392,Sat Mar 07 18:19:49 +0000 2020,People will stay hating cause they see you doi...,126,,"[😌, 🤷, 🏻, ♂, 😆]"
29818,1236355893496893441,Sat Mar 07 18:19:50 +0000 2020,RT @TheDoors: Happy Birthday to legendary arti...,148,"[ArthurLee, Love]",[]
29819,1236355895103229953,Sat Mar 07 18:19:50 +0000 2020,RT @SportingKC: @Chiefs 👊 Got another trophy f...,63,,[👊]


# process the text 

## cleaning and normalize

In [14]:
### define the function of tokenize and normalize 

#@Tokenize
def spacy_tokenize(string):
    tokens = []
    doc = nlp(string)
    for token in doc:
        tokens.append(token)
    return tokens

#@Normalize
def normalize(tokens):
    normalized_tokens = []
    for token in tokens:
        normalized = token.text.lower().strip()
        if ((token.is_alpha or token.is_digit)):
            normalized_tokens.append(normalized)
    return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
    return normalize(spacy_tokenize(string))


In [15]:
### tokenization and normalize text
text_list=df.iloc[:,2]
normalized_textlist=[]
for line in text_list:
    normalized_textlist.append(tokenize_normalize(line))
df['cleaned text']=normalized_textlist

In [16]:
### normalize the hash tags
tags_list=df.iloc[:,4]
normalized_tags_list=[]
for tags in tags_list:
    normalized_tags=[]
    for tag in tags:
        normalized_tag=tokenize_normalize(tag)
        if normalized_tag:
            normalized_tags.append(normalized_tag[0])
        else:
            normalized_tags.append('')
    normalized_tags_list.append(normalized_tags)
df['hash tags']=normalized_tags_list

## lemmazatization

In [17]:
### define a function to get the TAG of each word in sentence
wnl = WordNetLemmatizer()
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [18]:
### processing and lemmatize the words in sentences 
lemmatized_list=[]
for sentence in normalized_textlist:
    tagged_sent = pos_tag(sentence)
    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) 
    lemmatized_list.append(lemmas_sent)    


In [19]:
### load cleaned text into dataframe and double check if it worked
df['cleaned text']=lemmatized_list
df

,id,time,text,length,hash tags,emoji,cleaned text
0,1236305964870938624,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[happy, taeyeon, 태연, 소녀시대, girlsgeneration, 태연...",[🎀],"[rt, taeyeon, 태연, happy, photoshoot, behind, t..."
1,1236305965395234817,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[happy, taeyeon, 태연, 소녀시대, girlsgeneration, 태연...",[🎀],"[rt, taeyeon, 태연, happy, photoshoot, behind, t..."
2,1236305965546213376,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[happy, taeyeon, 태연, 소녀시대, girlsgeneration, 태연...",[🎀],"[rt, taeyeon, 태연, happy, photoshoot, behind, t..."
3,1236305965718417417,Sat Mar 07 15:01:26 +0000 2020,RT @missradders: Someone needs to hold him to ...,74,[],[👗],"[rt, someone, need, to, hold, him, to, this, i..."
4,1236305966292926466,Sat Mar 07 15:01:26 +0000 2020,RT @espn: MEAN MUG BRON 😤\n\nLeBron has been u...,111,[],[😤],"[rt, mean, mug, bron, lebron, have, be, up, to..."
...,...,...,...,...,...,...,...
29816,1236355889751343107,Sat Mar 07 18:19:49 +0000 2020,RT @SteveChopz: Found someone special in Idivi...,77,[],[😆],"[rt, find, someone, special, in, idivisible]"
29817,1236355889998651392,Sat Mar 07 18:19:49 +0000 2020,People will stay hating cause they see you doi...,126,[],"[😌, 🤷, 🏻, ♂, 😆]","[people, will, stay, hat, cause, they, see, yo..."
29818,1236355893496893441,Sat Mar 07 18:19:50 +0000 2020,RT @TheDoors: Happy Birthday to legendary arti...,148,"[arthurlee, love]",[],"[rt, happy, birthday, to, legendary, artist, a..."
29819,1236355895103229953,Sat Mar 07 18:19:50 +0000 2020,RT @SportingKC: @Chiefs 👊 Got another trophy f...,63,[],[👊],"[rt, get, another, trophy, for, kc, on, the, m..."


## eliminate the duplication

In [20]:
### creating a fake list to put all tokens in 
processing_cleaned_list=[]
for words in df.iloc[:,6]:
    text=''
    for word in words:
        text=text+' '+word
    processing_cleaned_list.append(text)
df['processing index']=processing_cleaned_list

In [21]:
# eliminate the dupilication by indentifying same fake string
df.drop_duplicates(subset='processing index',keep='first',inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(labels='processing index',axis=1,inplace=True)

In [22]:
# double check the data frame
df

,id,time,text,length,hash tags,emoji,cleaned text
0,1236305964870938624,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[happy, taeyeon, 태연, 소녀시대, girlsgeneration, 태연...",[🎀],"[rt, taeyeon, 태연, happy, photoshoot, behind, t..."
1,1236305965718417417,Sat Mar 07 15:01:26 +0000 2020,RT @missradders: Someone needs to hold him to ...,74,[],[👗],"[rt, someone, need, to, hold, him, to, this, i..."
2,1236305966292926466,Sat Mar 07 15:01:26 +0000 2020,RT @espn: MEAN MUG BRON 😤\n\nLeBron has been u...,111,[],[😤],"[rt, mean, mug, bron, lebron, have, be, up, to..."
3,1236305966846500868,Sat Mar 07 15:01:26 +0000 2020,RT @Sy_JOYstar: 180512 HQ\nEvery day missing ...,111,"[조이, joy, 박수영, 레드벨벳, redvelvet]",[💚],"[rt, 180512, hq, every, day, miss, you, 조이, jo..."
4,1236305968138518530,Sat Mar 07 15:01:26 +0000 2020,@goonermarky1 Big 💚 to you &amp; the #Family 👊...,69,[family],"[💚, 👊, 🍻]","[big, to, you, amp, the, family, have, a, grea..."
...,...,...,...,...,...,...,...
10660,1236355863507603458,Sat Mar 07 18:19:42 +0000 2020,Follow me on Instagram @alijeet11 \n|\n|\n|\n|...,137,"[poetry, love, poetrycommunity, writersofinsta...",[],"[follow, me, on, instagram, poetry, love, poet..."
10661,1236355872160243712,Sat Mar 07 18:19:44 +0000 2020,It's the #sight of it that will stop your #kid...,119,"[sight, kids]",[],"[it, the, sight, of, it, that, will, stop, you..."
10662,1236355873997344768,Sat Mar 07 18:19:45 +0000 2020,Inglewood Dad Caps ❤️\nNow Available https://t...,135,"[inglewood, theforum, randysdonuts, souvenir, ...",[❤],"[inglewood, dad, cap, now, available, inglewoo..."
10663,1236355884638523393,Sat Mar 07 18:19:47 +0000 2020,Lol the comments.\n\nThought they had a diamon...,63,[],[😆],"[lol, the, comment, think, they, have, a, diam..."


## correct spelling and drop off stopwords

In [23]:
#correted_textlist=[]
#for line in normalized_textlist:
    #ew_line=[]
    #for word in line:
        #blob=TextBlob(word)
        #new_line.append(str(blob.correct()))
    #correted_textlist.append(new_line)

In [24]:
#df['cleaned text']=correted_textlist
#for words in textlist:
    #if words in stopwords.words('english'):
        #words.remove(word)

# labelling

## define the class for hashtag

In [25]:
hashtag_classes=[["happy","glad","love","cheer", "enjoy", "joy","happiness","smile"],
         ["comfort","confident","relax","content","satisfied"],
         ["exciting", "excited",'amazing'],
         ["sad", "tense", "frustration","distressed"],
         ["afraid", "alarm", "anxiety", "concerned", "disgust","horrible","panic"],
         ["angry", "annoyed", "bear","mood"]]

## mark the hashtags

In [26]:
class_marks_hashtag=[[],[],[],[],[],[]]
processing_list_hashtags=df.iloc[:,4]
for tags in processing_list_hashtags:
    for i in range(0,len(hashtag_classes)):
        set_processing= set(tags)&set(hashtag_classes[i])
        if set_processing:
            class_marks_hashtag[i].append(len(set_processing))
        else:
            class_marks_hashtag[i].append(0)

## define emoji classes

In [27]:
emoji_classes=[[emoji.emojize(':laughing:',use_aliases=True),emoji.emojize(':smile:',use_aliases=True),emoji.emojize(':kissing_heart:',use_aliases=True),emoji.emojize(':heart:',use_aliases=True),emoji.emojize(':grin:',use_aliases=True)],
               [emoji.emojize(':blush:',use_aliases=True),emoji.emojize(':relaxed:',use_aliases=True)],
               [emoji.emojize(':exclamation:',use_aliases=True), emoji.emojize(':punch:',use_aliases=True),emoji.emojize(':heart_eyes:',use_aliases=True)],
               [emoji.emojize(':cry:',use_aliases=True),emoji.emojize(':sob:',use_aliases=True),emoji.emojize(':broken_heart:',use_aliases=True)],
               [emoji.emojize(':scream:',use_aliases=True), emoji.emojize(':fearful:',use_aliases=True), emoji.emojize(':worried:',use_aliases=True)],
               [emoji.emojize(':rage:',use_aliases=True), emoji.emojize(':angry:',use_aliases=True), emoji.emojize(':triumph:',use_aliases=True),emoji.emojize(':imp:',use_aliases=True)]]                                         

In [28]:
emoji_classes

[['😆', '😄', '😘', '❤', '😁'],
 ['😊', '☺'],
 ['❗', '👊', '😍'],
 ['😢', '😭', '💔'],
 ['😱', '😨', '😟'],
 ['😡', '😠', '😤', '👿']]

## mark the emojis

In [29]:
class_marks_emoji=[[],[],[],[],[],[]]
processing_list_emoji=df.iloc[:,5]
for tags in processing_list_emoji:
    for i in range(0,len(emoji_classes)):
        set_processing= set(tags)&set(emoji_classes[i])
        if set_processing:
            class_marks_emoji[i].append(0.5*len(set_processing))
        else:
            class_marks_emoji[i].append(0)

## load mark into dataframe

In [30]:
class_marks=[[],[],[],[],[],[]]
for i in range(0,len(class_marks)):
    class_marks[i]=np.sum([class_marks_emoji[i],class_marks_hashtag[i]],axis=0)

In [31]:
df['happy']=class_marks[0]
df['positive']=class_marks[1]
df['excitement']=class_marks[2]
df['surprise']=class_marks[3]
df['fear']=class_marks[4]
df['angry']=class_marks[5]
df

,id,time,text,length,hash tags,emoji,cleaned text,happy,positive,excitement,surprise,fear,angry
0,1236305964870938624,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[happy, taeyeon, 태연, 소녀시대, girlsgeneration, 태연...",[🎀],"[rt, taeyeon, 태연, happy, photoshoot, behind, t...",1.0,0.0,0.0,0.0,0.0,0.0
1,1236305965718417417,Sat Mar 07 15:01:26 +0000 2020,RT @missradders: Someone needs to hold him to ...,74,[],[👗],"[rt, someone, need, to, hold, him, to, this, i...",0.0,0.0,0.0,0.0,0.0,0.0
2,1236305966292926466,Sat Mar 07 15:01:26 +0000 2020,RT @espn: MEAN MUG BRON 😤\n\nLeBron has been u...,111,[],[😤],"[rt, mean, mug, bron, lebron, have, be, up, to...",0.0,0.0,0.0,0.0,0.0,0.5
3,1236305966846500868,Sat Mar 07 15:01:26 +0000 2020,RT @Sy_JOYstar: 180512 HQ\nEvery day missing ...,111,"[조이, joy, 박수영, 레드벨벳, redvelvet]",[💚],"[rt, 180512, hq, every, day, miss, you, 조이, jo...",1.0,0.0,0.0,0.0,0.0,0.0
4,1236305968138518530,Sat Mar 07 15:01:26 +0000 2020,@goonermarky1 Big 💚 to you &amp; the #Family 👊...,69,[family],"[💚, 👊, 🍻]","[big, to, you, amp, the, family, have, a, grea...",0.0,0.0,0.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10660,1236355863507603458,Sat Mar 07 18:19:42 +0000 2020,Follow me on Instagram @alijeet11 \n|\n|\n|\n|...,137,"[poetry, love, poetrycommunity, writersofinsta...",[],"[follow, me, on, instagram, poetry, love, poet...",1.0,0.0,0.0,0.0,0.0,0.0
10661,1236355872160243712,Sat Mar 07 18:19:44 +0000 2020,It's the #sight of it that will stop your #kid...,119,"[sight, kids]",[],"[it, the, sight, of, it, that, will, stop, you...",0.0,0.0,0.0,0.0,0.0,0.0
10662,1236355873997344768,Sat Mar 07 18:19:45 +0000 2020,Inglewood Dad Caps ❤️\nNow Available https://t...,135,"[inglewood, theforum, randysdonuts, souvenir, ...",[❤],"[inglewood, dad, cap, now, available, inglewoo...",1.5,0.0,0.0,0.0,0.0,0.0
10663,1236355884638523393,Sat Mar 07 18:19:47 +0000 2020,Lol the comments.\n\nThought they had a diamon...,63,[],[😆],"[lol, the, comment, think, they, have, a, diam...",0.5,0.0,0.0,0.0,0.0,0.0


## find the max mark and label the text

In [32]:
### loop the mark and find the best to label
max_list=[]
labels=[]
for i in range(0,len(df)):
    max=0
    order=0
    for j in range(0,len(class_marks)):
        if class_marks[j][i]>max:
            max=class_marks[j][i]
            order=j
    max_list.append(max)
    if order==0:
        labels.append('happy')
    elif order==1:
        labels.append('positive')
    elif order==2:
        labels.append('excitement')
    elif order==3:
        labels.append('surprise')
    elif order==4:
        labels.append('fear')
    elif order==5:
        labels.append('angry')
        
df['max_mark']=max_list
df['label']=labels    

In [33]:
df.reset_index(drop=True,inplace=True)
df

,id,time,text,length,hash tags,emoji,cleaned text,happy,positive,excitement,surprise,fear,angry,max_mark,label
0,1236305964870938624,Sat Mar 07 15:01:26 +0000 2020,RT @GirlsGeneration: TAEYEON 태연 '#Happy' Photo...,138,"[happy, taeyeon, 태연, 소녀시대, girlsgeneration, 태연...",[🎀],"[rt, taeyeon, 태연, happy, photoshoot, behind, t...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,happy
1,1236305965718417417,Sat Mar 07 15:01:26 +0000 2020,RT @missradders: Someone needs to hold him to ...,74,[],[👗],"[rt, someone, need, to, hold, him, to, this, i...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,happy
2,1236305966292926466,Sat Mar 07 15:01:26 +0000 2020,RT @espn: MEAN MUG BRON 😤\n\nLeBron has been u...,111,[],[😤],"[rt, mean, mug, bron, lebron, have, be, up, to...",0.0,0.0,0.0,0.0,0.0,0.5,0.5,angry
3,1236305966846500868,Sat Mar 07 15:01:26 +0000 2020,RT @Sy_JOYstar: 180512 HQ\nEvery day missing ...,111,"[조이, joy, 박수영, 레드벨벳, redvelvet]",[💚],"[rt, 180512, hq, every, day, miss, you, 조이, jo...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,happy
4,1236305968138518530,Sat Mar 07 15:01:26 +0000 2020,@goonermarky1 Big 💚 to you &amp; the #Family 👊...,69,[family],"[💚, 👊, 🍻]","[big, to, you, amp, the, family, have, a, grea...",0.0,0.0,0.5,0.0,0.0,0.0,0.5,excitement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10660,1236355863507603458,Sat Mar 07 18:19:42 +0000 2020,Follow me on Instagram @alijeet11 \n|\n|\n|\n|...,137,"[poetry, love, poetrycommunity, writersofinsta...",[],"[follow, me, on, instagram, poetry, love, poet...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,happy
10661,1236355872160243712,Sat Mar 07 18:19:44 +0000 2020,It's the #sight of it that will stop your #kid...,119,"[sight, kids]",[],"[it, the, sight, of, it, that, will, stop, you...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,happy
10662,1236355873997344768,Sat Mar 07 18:19:45 +0000 2020,Inglewood Dad Caps ❤️\nNow Available https://t...,135,"[inglewood, theforum, randysdonuts, souvenir, ...",[❤],"[inglewood, dad, cap, now, available, inglewoo...",1.5,0.0,0.0,0.0,0.0,0.0,1.5,happy
10663,1236355884638523393,Sat Mar 07 18:19:47 +0000 2020,Lol the comments.\n\nThought they had a diamon...,63,[],[😆],"[lol, the, comment, think, they, have, a, diam...",0.5,0.0,0.0,0.0,0.0,0.0,0.5,happy


## delete the potential ambiguous text and the text without right hashtag

In [34]:
final_df=df
final_df=final_df[final_df.max_mark!=0]

In [35]:
len(final_df[final_df.max_mark==1])

2226

In [36]:
#df.drop(labels='max mark',axis=1,inplace=True)

In [37]:
final_df.reset_index(drop=True,inplace=True)

#  output csv file

## analyze the number of the data

In [38]:
print('total numbers: ',len(final_df))
print('happy: ',len(final_df[final_df.label=='happy']))
print('angry: ',len(final_df[final_df.label=='angry']))
print('positive: ',len(final_df[final_df.label=='positive']))
print('excitement: ',len(final_df[final_df.label=='excitement']))
print('surprise: ',len(final_df[final_df.label=='surprise']))
print('fear: ',len(final_df[final_df.label=='fear']))

total numbers:  8880
happy:  3837
angry:  1790
positive:  476
excitement:  788
surprise:  1788
fear:  201


## load data into 6 different classes

In [39]:
csv_happy=final_df[final_df.label=='happy']
csv_angry=final_df[final_df.label=='angry']
csv_positive=final_df[final_df.label=='positive']
csv_excitement=final_df[final_df.label=='excitement']
csv_surprise=final_df[final_df.label=='surprise']
csv_fear=final_df[final_df.label=='fear']

## output the data into csv file

In [40]:
csv_happy.to_csv("csv_happy.csv",encoding='utf-8',index=False)
csv_angry.to_csv("csv_angry.csv",encoding='utf-8',index=False)
csv_positive.to_csv("csv_positive.csv",encoding='utf-8',index=False)
csv_excitement.to_csv("csv_excitement.csv",encoding='utf-8',index=False)
csv_surprise.to_csv("csv_surprise.csv",encoding='utf-8',index=False)
csv_fear.to_csv("csv_fear.csv",encoding='utf-8',index=False)

In [41]:
#final_df.to_csv("fourth_csv.csv",encoding='utf-8',index=False)

# csv processing

In [49]:
final_df.to_csv("all.csv",encoding='utf-8',index=False)